In [1]:
# import the necessary packages
from fastai.torch_core import flatten_model
from fastai.layers import CrossEntropyFlat
from fastai.vision import *
from fastai import *

In [2]:
# util functions
def arch_summary(arch):
    model = arch(False)
    tot = 0
    for i, l in enumerate(model.children()):
        n_layers = len(flatten_model(l))
        tot += n_layers
        print(f'({i}) {l.__class__.__name__:<12}: {n_layers:<4}layers (total: {tot})')
        
def get_groups(model, layer_groups):
    group_indices = [len(g) for g in layer_groups]
    curr_i = 0
    group = []
    for layer in model:
        group_indices[curr_i] -= len(flatten_model(layer))
        group.append(layer.__class__.__name__)
        if group_indices[curr_i] == 0:
            curr_i += 1
            print(f'Group {curr_i}:', group)   
            group = []
                
class FakeData:
    def __init__(self):
        self.c = 2
        self.path = './temp'    
        self.device = None
        self.loss_func = CrossEntropyFlat(axis=1)

In [2]:
data = ImageDataBunch.from_folder(path = "/home/varun/datasets/image_classification/cifar_10/32", train = "train", valid = "test",
                                 size = 32, ds_tfms = None, bs = 128)
data.normalize(imagenet_stats);

In [6]:
??imagenet_stats

Type:        tuple
String form: ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
Length:      2
Docstring:  
tuple() -> empty tuple
tuple(iterable) -> tuple initialized from iterable's items

If the argument is a tuple, the return value is the same object.


In [ ]:
data.show_batch(rows = 5, figsize = (10, 8))

In [4]:
learn = cnn_learner(data, models.resnet18, metrics = [accuracy])
opt = optim.SGD(learn.model[1].parameters(), lr = 0.01, momentum = 0.9, nesterov = True)
learn.opt_func = opt

In [ ]:
get_groups(nn.Sequential(*learn.model[0], *learn.model[1]), learn.layer_groups)

In [5]:
learn.fit_one_cycle(5, 0.01)

epoch,train_loss,valid_loss,accuracy,time
0,1.306405,1.170907,0.601900,00:32
1,0.965470,1.001598,0.653200,00:31
2,0.820377,0.810183,0.719900,00:31
3,0.746176,0.719907,0.750200,00:31
4,0.655457,0.702119,0.756800,00:31
